In [1]:
import datetime
import numpy as np
import talib as ta
import alpaca_trade_api as tradeapi
import time
import sys
sys.path.insert(0, '../')
import pandas
from datetime import timedelta

In [2]:
from config_paper import paper_key_id, paper_secret_key
from cross_signal import *

In [3]:
#The url at the end is what makes a paper trading bot
base_url='https://paper-api.alpaca.markets'
api = tradeapi.REST(paper_key_id, paper_secret_key,base_url)

In [4]:
startdate = datetime.datetime.today().strftime("%Y-%m-%d")
startdate

'2020-06-08'

In [5]:
#make sure to import these packages
from pandas import DataFrame
import talib.abstract as tab
#abstract automatically extracts the columns and periods from the data itself, no input required



#feel free to add or subtract more indicators for any other strategy you want to use
def populateindicators(dataframe) -> DataFrame:

        #momentum indicators
        #dataframe['willr'] = tab.WILLR(dataframe['high'].values, dataframe['low'].values, dataframe['close'].values)
        dataframe['ultosc'] = tab.ULTOSC(dataframe)
        
        
        dataframe['mfi'] = ta.MFI(dataframe['high'], dataframe['low'], dataframe['close'], np.asarray(dataframe['volume'], dtype='float'), timeperiod=14)
        
        dataframe['adx'] = tab.ADX(dataframe)

        dataframe['rsi'] = tab.RSI(dataframe, timeperiod=14)
        dataframe['rocr'] =tab.ROCR(dataframe)
        
        
        dataframe['plusdi'] =tab.PLUS_DI(dataframe)
        dataframe['minusdi'] =tab.MINUS_DI(dataframe)
        
        
        
        
        
        return dataframe

In [6]:
import statistics as sts
def quotelister(quotes):
        #exponential moving averages
        asksizelist=[]
        bidsizelist=[]
        askpricelist=[]
        bidpricelist=[]
        
        
        for quote in quotes:
            if quote.ask_size<50:
                if quote.bid_size<50:
                    asksizelist.append(quote.ask_size)
                    bidsizelist.append(quote.bid_size)
                    bidpricelist.append(quote.bid_size*quote.bid_price)
                    askpricelist.append(quote.ask_size*quote.ask_price)

        # required for graphing
        asksize=sum(asksizelist)
        bidsize=sum(bidsizelist)
        avgaskprice=round(sum(askpricelist)/asksize,2)
        avgbidprice=round(sum(bidpricelist)/bidsize,2)

        return asksize,bidsize,avgbidprice,avgaskprice

In [7]:
account = api.get_account()
print(account.status)
account

ACTIVE


Account({   'account_blocked': False,
    'account_number': 'PA2IRTBVTAQ6',
    'buying_power': '67356.834',
    'cash': '19690.29',
    'created_at': '2020-05-29T15:06:14.193215Z',
    'currency': 'USD',
    'daytrade_count': 77,
    'daytrading_buying_power': '67356.834',
    'equity': '30772.79',
    'id': '677d92c2-359d-4bd6-98a3-c3600c777156',
    'initial_margin': '5541.25',
    'last_equity': '29999.95',
    'last_maintenance_margin': '10399.239',
    'long_market_value': '11082.5',
    'maintenance_margin': '3324.75',
    'multiplier': '4',
    'pattern_day_trader': False,
    'portfolio_value': '30772.79',
    'regt_buying_power': '50463.08',
    'short_market_value': '0',
    'shorting_enabled': True,
    'sma': '0',
    'status': 'ACTIVE',
    'trade_suspended_by_user': False,
    'trading_blocked': False,
    'transfers_blocked': False})

In [8]:
with open('SP500_2020Q2SYM.txt', 'r') as f:
    symbols= f.read().split('\n')

In [9]:
symbollist=pd.read_csv('nasdaq-100-index-05-06-2020.csv')['Symbol']

In [10]:
barTimeframe = "1Min" # 1Min, 5Min, 15Min, 1H, 1D
assetsToTrade = symbollist
#assetsToTrade = [x.upper() for x in symbols]
positionSizing = 0.05

# Tracks position in list of symbols to download 
assetListLen = len(assetsToTrade)
trade=True

#create dict of buyorder.id, submitted_at
startorder_dict_list=[]


In [11]:
#get starting order by order id, see datetime and get rid
#create corresponding orders

In [ ]:
trade=True
while trade==True:
    #
    currenthour=time.localtime().tm_hour
    currentminute=time.localtime().tm_min
    currenttime=currenthour*100+currentminute
    currenttimestamp=api.get_clock().timestamp
            
            
    #Manage dict  
    if currenttime <=1230 and currenttime >=650: 
        orderlist=api.list_orders()
        expired_order_list=[]
        
        for i in range(len(startorder_dict_list)):
            if startorder_dict_list[i]['datetime']+datetime.timedelta(seconds=15)<currenttimestamp:
                expired_order_list.append(i)
                try:
                    api.cancel_order(startorder_dict_list[i]['id'])
                    print('cancelled '+startorder_dict_list[i]['symbol'] )
                except:('Fail cancellation')
        
        #remove expired orders after cancellation or expiration
        indexes = expired_order_list
        for index in sorted(indexes, reverse=True):
            del startorder_dict_list[index]
        
        
        
        
        
        for i in range(assetListLen) :
            try:
                symbol = assetsToTrade[i]

                returned_data = api.polygon.historic_agg_v2(symbol, 1, 'minute',_from=startdate, to=startdate).df
                    #api.get_barset(symbol,barTimeframe,limit=150).df
                data=populateindicators(returned_data)
                quoten=api.polygon.last_quote(symbol)
                buyprice=quoten.bidprice
            except:
                print('Issue with data '+ symbol)

        
            
            if data['adx'][-1] > 22 and \
            data['mfi'][-1] < 25 and \
            data['ultosc'][-1] < 25 and \
            data['rsi'][-1] < 33 and \
            data['minusdi'][-1] > 25 and \
            data['minusdi'][-1] > data['plusdi'][-1]:
                buyord=True
                assetlist=api.list_positions()
                for asset in assetlist:
                    # if asset is in the list of positions we have already bought, then do not buy again
                    if asset.symbol == symbol:
                        buyord=False
                        print(asset.symbol,asset.qty)

                #if asset is not already bought, then buy             
                if buyord==True:
                    #cashBalance = account.daytrading_buying_power
                    #equity
                    cashBalance = float(account.buying_power)
                    targetPositionSize = int(cashBalance / (data['close'][-1] / positionSizing))
                    buyreturned = api.submit_order(symbol,targetPositionSize,"buy","limit","day",limit_price=buyprice)
                    startorder_dict_list.append({'symbol':buyreturned.symbol,'id':buyreturned.id,'datetime':buyreturned.submitted_at,'side':buyreturned.side})
                    print('Buy ',buyreturned)


                
         

        
    
    time.sleep(3)
    #get current time and shut off trading after 12
     
    if currenthour >= 13: 
        trade=False
        print('Good work, Turning off')
    

    
    

Buy  Order({   'asset_class': 'us_equity',
    'asset_id': '31a9ec05-bf6a-4e77-8fbe-eafe432eab4a',
    'canceled_at': None,
    'client_order_id': 'a137a151-dbac-453e-af41-2e99c0804781',
    'created_at': '2020-06-08T15:04:06.065541Z',
    'expired_at': None,
    'extended_hours': False,
    'failed_at': None,
    'filled_at': None,
    'filled_avg_price': None,
    'filled_qty': '0',
    'id': 'a74c201d-d754-4369-85d0-a87398740707',
    'legs': None,
    'limit_price': '85.42',
    'order_class': '',
    'order_type': 'limit',
    'qty': '39',
    'replaced_at': None,
    'replaced_by': None,
    'replaces': None,
    'side': 'buy',
    'status': 'accepted',
    'stop_price': None,
    'submitted_at': '2020-06-08T15:04:05.975907Z',
    'symbol': 'DVA',
    'time_in_force': 'day',
    'type': 'limit',
    'updated_at': '2020-06-08T15:04:06.065541Z'})
Buy  Order({   'asset_class': 'us_equity',
    'asset_id': '31a9ec05-bf6a-4e77-8fbe-eafe432eab4a',
    'canceled_at': None,
    'client_o

In [ ]:
api.polygon.last_quote('SE').bidprice